# Testing para la web

In [ ]:
import paho.mqtt.client as mqtt
import serial, keyboard, json, random, time
from datetime import datetime


def generate_random_sensor_data():
    data = {        
        "Temperature": 27.5, # round(random.uniform(0, 100), 2),
        "Humidity": 50, #  random.randint(0, 100),
        "GasDetection": 200, # random.randint(0, 1),
        "FireDetection": 200, #random.randint(0, 1),
        "GardenLightStatus": True, # bool(random.getrandbits(1)),
        "GardenSoilHumidity": 90, # random.randint(0, 100),
        "ParkingCarStatus": True, # bool(random.getrandbits(1)),
        "ParkingAccessStatus": False, # bool(random.getrandbits(1)),
        "ParkingLightStatus": True # bool(random.getrandbits(1))
    }
    return json.dumps(data, indent=4)



# Configurar cliente MQTT
client = mqtt.Client()
client.connect("localhost", 1883, 10)

# Función para generar y publicar el JSON
def generate_and_publish_sensor_data():
    # Generar datos aleatorios utilizando la función generate_random_sensor_data()
    json_data = generate_random_sensor_data()
    print("JSON generado:", json_data)
    
    # Analizar el JSON generado
    try:
        sensor_data = json.loads(json_data)
        
        # Recorrer los elementos del JSON y publicarlos en MQTT
        for key, value in sensor_data.items():
            topic = key
            payload = str(value)  # Convertir el valor a cadena si es necesario
            client.publish(topic, payload)
            # print(f"Publicado en MQTT - Tema: {topic}, Valor: {payload}")
            
        # Puedes procesar el JSON aquí según tus necesidades
    except json.JSONDecodeError as e:
        print("Error al decodificar JSON:", e)


generate_and_publish_sensor_data()
    

In [ ]:
while True:
    if keyboard.is_pressed('q'):
        print("Se presionó 'q', saliendo...")
        break  # Romper el bucle
    
    # Llamar a la función para generar y publicar el JSON
    generate_and_publish_sensor_data()
    
    # Esperar 1 segundo antes de la siguiente iteración
    time.sleep(1)


# Lectura del puerto serial Arduino

In [1]:
import serial
import paho.mqtt.client as mqtt
import keyboard
import json

# Configurar la conexión serie
arduino = serial.Serial('COM4', 9600)

# Configurar cliente MQTT
client = mqtt.Client()
client.connect("localhost", 1883, 10)

while True:
    if keyboard.is_pressed('q'):
        print("Se presionó 'q', saliendo...")
        break  # Romper el bucle
    
    # Leer datos del puerto serial
    serial_data = arduino.readline().decode().strip()

    # Verificar si los datos comienzan con "{" (inicio del JSON)
    if serial_data.startswith("{"):
        # Continuar leyendo hasta que se reciba un "}" (fin del JSON)
        while not serial_data.endswith("}"):
            serial_data += arduino.readline().decode().strip()
        
        # Analizar el JSON
        try:
            json_data = json.loads(serial_data)
            print("JSON recibido:", json_data)
            
            # Recorrer los elementos del JSON y publicarlos en MQTT
            for key, value in json_data.items():
                topic = key
                payload = str(value)  # Convertir el valor a cadena si es necesario
                client.publish(topic, payload)
                # print(f"Publicado en MQTT - Tema: {topic}, Valor: {payload}")
            
            # Puedes procesar el JSON aquí según tus necesidades
        except json.JSONDecodeError as e:
            print("Error al decodificar JSON:", e)


JSON recibido: {'Temperature': 25.5, 'Humidity': 50, 'GasDetection': 0, 'FireDetection': 0, 'GardenLightStatus': False, 'GardenSoilHumidity': 50, 'ParkingCarStatus': False, 'ParkingAccessStatus': False, 'ParkingLightStatus': False}
JSON recibido: {'Temperature': 25, 'Humidity': 58, 'GasDetection': 96, 'FireDetection': 73, 'GardenLightStatus': False, 'GardenSoilHumidity': 50, 'ParkingCarStatus': False, 'ParkingAccessStatus': False, 'ParkingLightStatus': False}
JSON recibido: {'Temperature': 25, 'Humidity': 58, 'GasDetection': 96, 'FireDetection': 73, 'GardenLightStatus': False, 'GardenSoilHumidity': 50, 'ParkingCarStatus': False, 'ParkingAccessStatus': True, 'ParkingLightStatus': False}
JSON recibido: {'Temperature': 25, 'Humidity': 58, 'GasDetection': 96, 'FireDetection': 73, 'GardenLightStatus': False, 'GardenSoilHumidity': 50, 'ParkingCarStatus': False, 'ParkingAccessStatus': True, 'ParkingLightStatus': False}
JSON recibido: {'Temperature': 25, 'Humidity': 58, 'GasDetection': 96, 'Fi

KeyboardInterrupt: 